<a href="https://colab.research.google.com/github/FarhanDhanani/IBCAST/blob/main/TABLE_QA_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
permToSampleTables = False

In [ ]:
# install datasets
%tensorflow_version 2.x
!pip install datasets
!pip install --upgrade langchain openai -q
#!pip install "pinecone-client[grpc]"
!pip install pinecone-client
!pip install transformers sentence_transformers torch-scatter
!pip install --upgrade protobuf

In [ ]:
import json
import time
import pinecone
import pandas as pd
from tqdm.auto import tqdm
from datasets import load_dataset
from langchain.embeddings.openai import OpenAIEmbeddings

import torch
from sentence_transformers import SentenceTransformer
#from transformers import TapasTokenizer
from transformers import pipeline, TapasTokenizer, TapasForQuestionAnswering

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
from google.colab import drive
from google.colab import files
base_path = '/content/drive'
drive.mount(base_path)
base_path = base_path + '/My Drive/'

base_path = base_path+"/dataset/TABLEQA/"

Mounted at /content/drive


In [ ]:
dataset = load_dataset("DongfuTingle/FeTaQA")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/DongfuTingle___json/DongfuTingle--FeTaQA-163770c1c4d4cdc5/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def sampleTables(datasetFileName, totalRecordsToSample):
  processedDataSet=[]
  for index in range(totalRecordsToSample):
    headers = dataset['train'][index]['table_array'][0]
    content = dataset['train'][index]['table_array'][1:]
    table = pd.DataFrame(content, columns=headers)

    rowChunks = table.to_dict('records')
    colChunks = table.to_dict('list')

    for col in colChunks:
      colChunk = colChunks[col]
      rowChunks.append({col:colChunk})

    rowChunks.append({'table_page_title' :dataset['train'][index]['table_page_title']})
    rowChunks.append({'table_section_title' :dataset['train'][index]['table_section_title']})

    wholeChunk = {
        dataset['train'][index]['table_page_title']
        + " " +
        dataset['train'][index]['table_section_title']:
        rowChunks
    }
    processedDataSet.append(wholeChunk)

  with open(base_path+datasetFileName, "w") as outfile:
      outfile.write(json.dumps(processedDataSet, indent = 4))


In [ ]:
datasetFileName = "TQA.json"
totalRecordsToSample = 50

if(permToSampleTables):
  sampleTables(datasetFileName, totalRecordsToSample)

In [ ]:
def _preprocess_tables(datasetFileName):
    with open(base_path+datasetFileName, 'r') as infile:
      processedDataSet = json.load(infile)

    tableChunks = []
    for wholeChunk in processedDataSet:
      for mainKey in wholeChunk:
        chunks = wholeChunk[mainKey]
        for chunk in chunks:
          chunk["record"] = mainKey
          tableChunks.append(json.dumps(chunk))
    return tableChunks

## **EMBEDDING**

In [ ]:
import openai
openai.api_key = "SECRET"
openai.Engine.list()

In [ ]:
pinecone.init(
    api_key="SECRET",
    environment="secret enviroment"
)

In [ ]:
MODEL = "text-embedding-ada-002"
embeder = openai.Embedding.create(
    input=["checking emeding"],
    engine=MODEL
)

In [ ]:
embeds = [record['embedding'] for record in embeder['data']]
embeds

In [ ]:
len(embeds[0])

1536

In [ ]:
index_name = "table-qa-openai"

if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=len(embeds[0]),
        metric="cosine"
    )

index = pinecone.Index(index_name)

In [ ]:
batch_size = 4
processed_tables = _preprocess_tables(datasetFileName)
for i in tqdm(range(0, len(processed_tables), batch_size)):

    i_end = min(i+batch_size, len(processed_tables))
    batch = processed_tables[i:i_end]
    res = openai.Embedding.create(input=batch, engine=MODEL)
    embeds = [record['embedding'] for record in res['data']]

    meta = [{'chunk': chunk} for chunk in batch]
    ids = [f"{idx}" for idx in range(i, i_end)]

    to_upsert = list(zip(ids, embeds, meta))
    _ = index.upsert(vectors=to_upsert)
    time.sleep(2)

# check that we have all vectors in index
index.describe_index_stats()

  0%|          | 0/264 [00:00<?, ?it/s]

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1054}},
 'total_vector_count': 1054}

## **QUESTION ANSWERING**

In [ ]:
retreivedResultsFileName = "retreivedResults.json"

In [ ]:
responses = []
for idx in tqdm(range(totalRecordsToSample)):
    query = dataset['train'][idx]['question']
    xq = openai.Embedding.create(input=query, engine=MODEL)['data'][0]['embedding']
    res = index.query([xq], top_k=5, include_metadata=True)
    retreivedResponse= {'question': query, 'retrievedResults': res.to_dict()}
    responses.append(retreivedResponse)
    time.sleep(2)

with open(base_path+retreivedResultsFileName, "w") as outfile:
    outfile.write(json.dumps(responses, indent = 4))


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
def answer_question(
    extractedRecords,
    question,
    model="text-davinci-003",
    max_len=1800,
    size="ada",
    debug=False,
    max_tokens=500,
    stop_sequence=None
):

    """
    Answer a question based on the most similar context from the dataframe texts
    """
    contextDocs = [ rec['metadata']['chunk'] for rec in retreivedResults[0]['retrievedResults']['matches'] ]
    # If debug, print the raw model response
    if debug:
        print("Context:\n" + contextDocs)
        print("\n\n")

    try:
        # Create a completions using the question and context
        response = openai.Completion.create(
            prompt=f"Answer the question based on the context below \n\nContext: {contextDocs}\n\n---\n\nQuestion: {question}\nAnswer:",
            temperature=0,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=stop_sequence,
            model=model,
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""


In [ ]:
with open(base_path+retreivedResultsFileName, 'r') as infile:
      retreivedResults = json.load(infile)

In [ ]:
sentenceLikeOutputsFileName = "senteceLikeOutputs2.json"

In [ ]:
outputs = []
for results in tqdm(retreivedResults):
  question = results['question']
  gen_ans = answer_question(results['retrievedResults'], question)
  output = {'question': question, 'gen_ans': gen_ans}
  outputs.append(output)
  time.sleep(2)

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
with open(base_path+sentenceLikeOutputsFileName, "w") as outfile:
    outfile.write(json.dumps(outputs, indent = 4))

# EVALUATIONS

## TESTING PINECONE

In [ ]:
import warnings
warnings.filterwarnings("ignore")

with open(base_path+'retreivedResults.json', 'r') as infile:
      result = json.load(infile)

avg_precission = 0
avg_recall = 0

for index in range(50):
  cell_ids = dataset['train'][index]['highlighted_cell_ids']

  headers = dataset['train'][index]['table_array'][0]
  content = dataset['train'][index]['table_array'][1:]
  table = pd.DataFrame(content, columns=headers)

  factsRows = []
  factsCols = []

  rowIndexSet = set()
  colIndexSet = set()

  for cell_id in cell_ids:
    rowIndex = cell_id[0]-1
    colIndex = cell_id[1]
    record_sub = dataset['train'][index]['table_page_title'] + " "\
                  + dataset['train'][index]['table_section_title']

    if(not(rowIndex in rowIndexSet)):
      record_row = table.to_dict('records')[rowIndex]
      record_row['record'] = record_sub
      factsRows.append(record_row)
      rowIndexSet.add(rowIndex)

    if(not(colIndex in colIndexSet)):
      record_col = { \
                    headers[colIndex]:table.to_dict('list')[headers[colIndex]], \
                    'record':record_sub
                   }
      factsCols.append(record_col)
      colIndexSet.add(colIndex)


  matches = 0
  num_of_retrieved_records = 0
  num_of_relevant_recordds = len(factsRows)+len(factsCols)

  for record in result[index]['retrievedResults']['matches']:
    num_of_retrieved_records+=1

    for row in factsRows:
      if(json.loads(record['metadata']['chunk']).items() == row.items()):
        matches+=1


    for col in factsCols:
      if(json.loads(record['metadata']['chunk']).items() == col.items()):
        matches+=1


  precission = matches / num_of_retrieved_records * 100
  recall = matches / num_of_relevant_recordds * 100
  avg_precission += precission
  avg_recall += recall

In [ ]:
print("average precission =", avg_precission/50, "average recall =", avg_recall/50)

average precission = 50.4 average recall = 45.17002442002442


## TESTING OPEN AI TEXT DAVINCI

In [ ]:
with open(base_path+'senteceLikeOutputs2.json', 'r') as infile:
      outputs = json.load(infile)

In [ ]:
expected_answers = []
generated_answers = []
for index in range(50):
  expected_answers.append(dataset['train'][index]['answer'])
  generated_answers.append(outputs[index]['gen_ans'])

headers = ['Expected Answers', "Generated Answers"]
content = list(zip(expected_answers, generated_answers))
table = pd.DataFrame(content, columns=headers)

In [ ]:
table

,Expected Answers,Generated Answers
0,Thompson prevailed in the 1982 Illinois gubern...,James R. Thompson (incumbent) won the 1982 Ill...
1,Michael Andretti finished with a run of 214.52...,Michael and Mario Andretti did not participate...
2,"B'z The Best ""Pleasure"" sold more than 5 milli...",This question cannot be answered based on the ...
3,The Tigerair Australia fleet consists of the f...,This question cannot be answered based on the ...
4,"In 2016 and 2017, Platt played in Dear Evan Ha...",Platt did not appear in the 1982 Illinois gube...
5,EUTRAN has download rates of 299.6 Mbit/s and ...,This question is not related to the context pr...
6,Melina Kanakaredes played in television dramas...,Melina Kanakaredes did not play any roles in t...
7,"In the 2015–16 season, Austin Fyten spent with...",This question is not related to the context pr...
8,The World U-17 Hockey Challenge attracted U-17...,This question is not related to the context pr...
9,"After playing with Partizani, Hyshmeri played ...",This question is not related to the context pr...
